<a href="https://colab.research.google.com/github/MiTi-alt/ML_TV201/blob/main/Lab_6_20130434_TranDinhMinhTien.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This lab is to deal with classification task using **Random Forests** and **Naïve Bayes** algorithms with/without **Feature Selection**. 

*   **Deadline: 23:59, 25/03/2023**



# Import libraries

In [4]:
# code
from sklearn.datasets import fetch_openml
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from prettytable import PrettyTable
from sklearn.datasets import load_iris
from sklearn import datasets
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ML_TH_2023'
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.metrics import average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/ML_TH_2023


#Task 1. 
Task 1. Compare the performance of selected classification algorithms including **Random forest**, **NaiveBayes**, and **SVM** with **mnist** dataset based on **accuracy, precision, recall, f1** measures according to **without using selection feature** and **using selection feature**.



In [ ]:
# code
mnist = datasets.load_digits()
X = mnist.data
y = mnist.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train, y_train)

nb = GaussianNB()
nb.fit(X_train, y_train)

svm = SVC()
svm.fit(X_train, y_train)
clf_pred = clf.predict(X_test)
clf_acc = accuracy_score(y_test, clf_pred)
clf_pre = precision_score(y_test, clf_pred, average='weighted')
clf_rec = recall_score(y_test, clf_pred, average='weighted')
clf_f1 = f1_score(y_test, clf_pred, average='weighted')

nb_pred = nb.predict(X_test)
nb_acc = accuracy_score(y_test, nb_pred)
nb_pre = precision_score(y_test, nb_pred, average='weighted')
nb_rec = recall_score(y_test, nb_pred, average='weighted')
nb_f1 = f1_score(y_test, nb_pred, average='weighted')

svm_pred = svm.predict(X_test)
svm_acc = accuracy_score(y_test, svm_pred)
svm_pre = precision_score(y_test, svm_pred, average='weighted')
svm_rec = recall_score(y_test, svm_pred, average='weighted')
svm_f1 = f1_score(y_test, svm_pred, average='weighted')

t = PrettyTable(["algorithms","Random Forest","Naive Bayes","Support Vector Machine"])
t.add_row(["Accuracy",clf_acc,nb_acc,svm_acc])
t.add_row(["Precision",clf_pre,nb_pre,svm_pre])
t.add_row(["Recall",clf_rec,nb_rec,svm_rec])
t.add_row(["F1 Score:",clf_f1,nb_f1,svm_f1])
print(t)



+------------+--------------------+--------------------+------------------------+
| algorithms |   Random Forest    |    Naive Bayes     | Support Vector Machine |
+------------+--------------------+--------------------+------------------------+
|  Accuracy  | 0.9638888888888889 | 0.8194444444444444 |   0.9805555555555555   |
| Precision  | 0.9659283564388323 | 0.8665616869790489 |   0.9811124551971326   |
|   Recall   | 0.9638888888888889 | 0.8194444444444444 |   0.9805555555555555   |
| F1 Score:  | 0.963758796002071  | 0.8226013409125894 |   0.9805406180148828   |
+------------+--------------------+--------------------+------------------------+


#Task 2. 
For given bank dataset (bank.csv) having the following attributes :
1.	**age** (numeric)
2.	**job** : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3.	**marital** : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4.	**education** (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5.	**default**: has credit in default? (categorical: 'no','yes','unknown')
6.	**housing**: has housing loan? (categorical: 'no','yes','unknown')
7.	**loan**: has personal loan? (categorical: 'no','yes','unknown')
8.	**contact**: contact communication type (categorical: 'cellular','telephone')
9.	**month**: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10.	**day_of_week**: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11.	**duration**: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
12.	**campaign**: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13.	**pdays**: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14.	**previous**: number of contacts performed before this campaign and for this client (numeric)
15.	**poutcome**: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
Output variable (desired target):
16.	**y**. has the client subscribed a term deposit? (binary: 'yes','no')



#3.1. Apply StandardScaler() function to columns that contains numerical data ('age', 'balance', 'day', 'campaign', 'pdays', 'previous')

In [20]:
#code
data = pd.read_csv("bank.csv")

numerical_cols = ["age", "balance", "day", "campaign", "pdays", "previous"]

scaler = StandardScaler()
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])
print(data[numerical_cols])

            age   balance       day  campaign     pdays  previous
0      1.491505  0.252525 -1.265746 -0.554168 -0.481184 -0.363260
1      1.239676 -0.459974 -1.265746 -0.554168 -0.481184 -0.363260
2     -0.019470 -0.080160 -1.265746 -0.554168 -0.481184 -0.363260
3      1.155733  0.293762 -1.265746 -0.554168 -0.481184 -0.363260
4      1.071790 -0.416876 -1.265746 -0.186785 -0.481184 -0.363260
...         ...       ...       ...       ...       ...       ...
11157 -0.691015 -0.473616  0.515650 -0.554168 -0.481184 -0.363260
11158 -0.187357 -0.246658  0.040612  0.547981 -0.481184 -0.363260
11159 -0.774958 -0.464934  0.396891 -0.186785 -0.481184 -0.363260
11160  0.148416 -0.473926 -0.909466 -0.186785  1.109571  1.818332
11161 -0.607072 -0.473926 -0.790707 -0.554168 -0.481184 -0.363260

[11162 rows x 6 columns]


#3.2. Apply Encode Categorical Value (OneHotEncoder) to transfrom categorical data to numerical data ('job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome')

In [23]:
#code
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
onehotencoder = OneHotEncoder()
onehot_encoded = onehotencoder.fit_transform(data[cat_cols]).toarray()
onehot_encoded_df = pd.DataFrame(onehot_encoded, columns = onehotencoder.get_feature_names_out(cat_cols))
data.drop(columns = cat_cols, inplace=True)
bank_data_encoded = pd.concat([data, onehot_encoded_df], axis=1)
print(bank_data_encoded)



            age   balance       day  duration  campaign     pdays  previous  \
0      1.491505  0.252525 -1.265746      1042 -0.554168 -0.481184 -0.363260   
1      1.239676 -0.459974 -1.265746      1467 -0.554168 -0.481184 -0.363260   
2     -0.019470 -0.080160 -1.265746      1389 -0.554168 -0.481184 -0.363260   
3      1.155733  0.293762 -1.265746       579 -0.554168 -0.481184 -0.363260   
4      1.071790 -0.416876 -1.265746       673 -0.186785 -0.481184 -0.363260   
...         ...       ...       ...       ...       ...       ...       ...   
11157 -0.691015 -0.473616  0.515650       257 -0.554168 -0.481184 -0.363260   
11158 -0.187357 -0.246658  0.040612        83  0.547981 -0.481184 -0.363260   
11159 -0.774958 -0.464934  0.396891       156 -0.186785 -0.481184 -0.363260   
11160  0.148416 -0.473926 -0.909466         9 -0.186785  1.109571  1.818332   
11161 -0.607072 -0.473926 -0.790707       628 -0.554168 -0.481184 -0.363260   

      deposit  job_admin.  job_blue-collar  ...  mo

#3.3. Apply **Decision tree, Random forest, kNN, NaïveBayes** to preproceed dataset in the previous steps. Then compare the obtained results using **accuracy, precision, recall, f1** measures.

In [40]:
#code
X = bank_data_encoded.drop('deposit', axis=1)
y = bank_data_encoded['deposit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

dt = DecisionTreeClassifier(random_state=8)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
acc_dt = accuracy_score(y_test, y_pred_dt)
prec_dt = precision_score(y_test, y_pred_dt, pos_label='yes')
rec_dt = recall_score(y_test, y_pred_dt, pos_label='yes')
f1_dt = f1_score(y_test, y_pred_dt, pos_label='yes')

rf = RandomForestClassifier(random_state=8)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf, pos_label='yes')
rec_rf = recall_score(y_test, y_pred_rf, pos_label='yes')
f1_rf = f1_score(y_test, y_pred_rf, pos_label='yes')

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
acc_knn = accuracy_score(y_test, y_pred_knn)
prec_knn = precision_score(y_test, y_pred_knn, pos_label='yes')
rec_knn = recall_score(y_test, y_pred_knn, pos_label='yes')
f1_knn = f1_score(y_test, y_pred_knn, pos_label='yes')

nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
acc_nb = accuracy_score(y_test, y_pred_nb)
prec_nb = precision_score(y_test, y_pred_nb,pos_label='yes')
rec_nb = recall_score(y_test, y_pred_nb,pos_label='yes')
f1_nb = f1_score(y_test, y_pred_nb,pos_label='yes')

t = PrettyTable(["algorithms","Decision Tree","Random Forest","KNN","Naive Bayes"])
t.add_row(["Accuracy",acc_dt,acc_rf,acc_knn,acc_nb])
t.add_row(["Precision",prec_dt,acc_rf,prec_knn,prec_nb])
t.add_row(["Recall",rec_dt,rec_rf,rec_knn,rec_nb])
t.add_row(["F1 Score:",f1_dt,f1_rf,f1_knn,f1_nb])
print(t)

+------------+--------------------+--------------------+--------------------+--------------------+
| algorithms |   Decision Tree    |   Random Forest    |        KNN         |    Naive Bayes     |
+------------+--------------------+--------------------+--------------------+--------------------+
|  Accuracy  | 0.8007165248544559 | 0.8607254814151366 |  0.77384684281236  | 0.7174205105239588 |
| Precision  | 0.784037558685446  | 0.8607254814151366 | 0.7602674307545367 | 0.7760210803689065 |
|   Recall   | 0.7952380952380952 | 0.9066666666666666 | 0.758095238095238  | 0.560952380952381  |
| F1 Score:  | 0.789598108747045  | 0.8595936794582393 | 0.759179780639008  | 0.6511885019347707 |
+------------+--------------------+--------------------+--------------------+--------------------+


#3.4. Using selection feature to above dataset, then compare the classification results with those in Task 2.3. 

In [72]:
#code

X = bank_data_encoded.drop('deposit', axis=1)
y = bank_data_encoded['deposit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

dt = DecisionTreeClassifier()
selector = RFE(dt, n_features_to_select=8)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

dt.fit(X_train_selected, y_train)
y_pred_dt = dt.predict(X_test_selected)
acc_dt = accuracy_score(y_test, y_pred_dt)
prec_dt = precision_score(y_test, y_pred_dt, pos_label='yes')
rec_dt = recall_score(y_test, y_pred_dt, pos_label='yes')
f1_dt = f1_score(y_test, y_pred_dt, pos_label='yes')

rf = RandomForestClassifier()
selector_rf = RFE(rf, n_features_to_select=8)
X_train_selected_rf = selector_rf.fit_transform(X_train, y_train)
X_test_selected_rf = selector_rf.transform(X_test)
rf.fit(X_train_selected_rf, y_train)
y_pred_rf = rf.predict(X_test_selected_rf)
acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf, pos_label='yes')
rec_rf = recall_score(y_test, y_pred_rf, pos_label='yes')
f1_rf = f1_score(y_test, y_pred_rf, pos_label='yes')

t = PrettyTable(["algorithms","Decision Tree","Random Forest"])
t.add_row(["Accuracy",acc_dt,acc_rf])
t.add_row(["Precision",prec_dt,acc_rf])
t.add_row(["Recall",rec_dt,rec_rf])
t.add_row(["F1 Score:",f1_dt,f1_rf])
print(t)

+------------+--------------------+--------------------+
| algorithms |   Decision Tree    |   Random Forest    |
+------------+--------------------+--------------------+
|  Accuracy  | 0.7429083308450284 | 0.8235294117647058 |
| Precision  | 0.724937343358396  | 0.8235294117647058 |
|   Recall   | 0.7327422419252692 | 0.8366054464851171 |
| F1 Score:  | 0.7288188976377953 | 0.8171976492421898 |
+------------+--------------------+--------------------+


#Task 4. 
For a given dataset in the Lab #5 (**credit card dataset**), perform feature selection and thencompare the performance of selected classification algorithms (Decision Tree, kNN, Logistic Regression, SVM, Random Forest and NaiveBayes) based on accuracy, precision, recall, f1 measures.


In [7]:
# code
file = pd.read_csv('creditcard.csv')
X = file.drop('Class', axis=1)
y = file['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
dtc = DecisionTreeClassifier(random_state=3)
dtc.fit(X_train, y_train)
y_pred_dtc = dtc.predict(X_test)
dtc_acc = accuracy_score(y_test, y_pred_dtc)
dtc_prec = precision_score(y_test, y_pred_dtc)
dtc_rec = recall_score(y_test, y_pred_dtc)
dtc_f1 = f1_score(y_test, y_pred_dtc)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
knn_acc = accuracy_score(y_test, y_pred_knn)
knn_prec = precision_score(y_test, y_pred_knn)
knn_rec = recall_score(y_test, y_pred_knn)
knn_f1 = f1_score(y_test, y_pred_knn)

lg = LogisticRegression(random_state=3)
lg.fit(X_train, y_train)
y_pred_lg = lg.predict(X_test)
lg_acc = accuracy_score(y_test, y_pred_lg)
lg_prec = precision_score(y_test, y_pred_lg)
lg_rec = recall_score(y_test, y_pred_lg)
lg_f1 = f1_score(y_test, y_pred_lg)



clf = svm.SVC(kernel='rbf', random_state=3)
clf.fit(X_train, y_train)
y_pred_clf = clf.predict(X_test)
clf_acc = accuracy_score(y_test, y_pred_clf)
clf_prec = precision_score(y_test, y_pred_clf)
clf_rec = recall_score(y_test, y_pred_clf)
clf_f1 = f1_score(y_test, y_pred_clf)

nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
acc_nb = accuracy_score(y_test, y_pred_nb)
prec_nb = precision_score(y_test, y_pred_nb)
rec_nb = recall_score(y_test, y_pred_nb)
f1_nb = f1_score(y_test, y_pred_nb)


t = PrettyTable(["algorithms","SVM","Logistic Regression","Decision Tree","kNN","Naive Bayes"])
t.add_row(["Accuracy",clf_acc,lg_acc,dtc_acc,knn_acc,acc_nb])
t.add_row(["Precision",clf_prec,lg_prec,dtc_prec,knn_prec,prec_nb])
t.add_row(["Recall",clf_rec,lg_rec,dtc_rec,knn_rec,rec_nb])
t.add_row(["F1 Score:",clf_f1,lg_f1,dtc_f1,knn_f1,f1_nb])
print(t)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


+------------+--------------------+---------------------+--------------------+---------------------+---------------------+
| algorithms |        SVM         | Logistic Regression |   Decision Tree    |         kNN         |     Naive Bayes     |
+------------+--------------------+---------------------+--------------------+---------------------+---------------------+
|  Accuracy  | 0.9982444436641972 |  0.9988062216916541 | 0.9992099996488887 |  0.9982971103542713 |  0.9919946631087392 |
| Precision  |        0.0         |  0.6568627450980392 | 0.8089887640449438 |  0.7142857142857143 | 0.11965811965811966 |
|   Recall   |        0.0         |         0.67        |        0.72        |         0.05        |         0.56        |
| F1 Score:  |        0.0         |  0.6633663366336634 | 0.761904761904762  | 0.09345794392523366 | 0.19718309859154928 |
+------------+--------------------+---------------------+--------------------+---------------------+---------------------+


#Finally,
Save a copy in your Github. Remember renaming the notebook.